In [1]:
!pip install bpy

  Using cached bpy-2.82.1-cp37-cp37m-win_amd64.whl (105.7 MB)


In [2]:
!pip install numpy

  Using cached numpy-1.21.6-cp37-cp37m-win_amd64.whl (14.0 MB)


In [1]:
import tkinter as tk
from tkinter import *
from tkinter import ttk
from tkinter import filedialog
import bpy
import os
import sys

In [2]:
class Process:
    def __init__(self, obj_file, target_faces, export_name):
        mesh = self.load_obj(obj_file)
        self.simplify(mesh, target_faces)
        self.export_obj(mesh, export_name)

    def load_obj(self, obj_file):
        bpy.ops.import_scene.obj(filepath=obj_file, axis_forward='-Z', axis_up='Y', filter_glob="*.obj;*.mtl", use_edges=True,
                                 use_smooth_groups=True, use_split_objects=False, use_split_groups=False,
                                 use_groups_as_vgroups=False, use_image_search=True, split_mode='ON')
        ob = bpy.context.selected_objects[0]
        return ob

    def subsurf(self, mesh):
        # subdivide mesh
        bpy.context.view_layer.objects.active = mesh
        mod = mesh.modifiers.new(name='Subsurf', type='SUBSURF')
        mod.subdivision_type = 'SIMPLE'
        bpy.ops.object.modifier_apply(modifier=mod.name)
        # now triangulate
        mod = mesh.modifiers.new(name='Triangluate', type='TRIANGULATE')
        bpy.ops.object.modifier_apply(modifier=mod.name)

    def simplify(self, mesh, target_faces):
        bpy.context.view_layer.objects.active = mesh
        mod = mesh.modifiers.new(name='Decimate', type='DECIMATE')
        bpy.context.object.modifiers['Decimate'].use_collapse_triangulate = True
        #
        nfaces = len(mesh.data.polygons)
        if nfaces < target_faces:
            self.subsurf(mesh)
            nfaces = len(mesh.data.polygons)
        ratio = target_faces / float(nfaces)
        mod.ratio = float('%s' % ('%.6g' % (ratio)))
        print('faces: ', mod.face_count, mod.ratio)
        bpy.ops.object.modifier_apply(modifier=mod.name)


    def export_obj(self, mesh, export_name):
        outpath = os.path.dirname(export_name)
        if not os.path.isdir(outpath): os.makedirs(outpath)
        print('EXPORTING', export_name)
        bpy.ops.object.select_all(action='DESELECT')
        mesh.select_set(state=True)
        bpy.ops.export_scene.obj(filepath=export_name, check_existing=False, filter_glob="*.obj;*.mtl",
                                 use_selection=True, use_animation=False, use_mesh_modifiers=True, use_edges=True,
                                 use_smooth_groups=False, use_smooth_groups_bitflags=False, use_normals=True,
                                 use_uvs=False, use_materials=False, use_triangles=True, use_nurbs=False,
                                 use_vertex_groups=False, use_blen_objects=True, group_by_object=False,
                                 group_by_material=False, keep_vertex_order=True, global_scale=1, path_mode='AUTO',
                                 axis_forward='-Z', axis_up='Y')



In [3]:
root = Tk(className='LBProcess')

def donothing():
   x = 0

def openfile():
    global file_path
    file_path = filedialog.askopenfilename(title="Select 3D Image")
    if file_path:
        file_name = file_path.split('/')
        model_name = file_name.pop()
        progressFrame.pack(fill="both")
        myprogress = ttk.Progressbar(progressFrame, orient= HORIZONTAL,length=300,mode='determinate')
        myprogress.pack(pady=20)
        myprogress['value'] += 100
        root.update_idletasks()
        obj_file = file_path
        target_faces = 19996
        export_name = '3D Models/' + model_name
        blender = Process(obj_file, target_faces, export_name)
        myprogress.stop()
        unpack_all_frames()
        clear_frame(labelFrame)
        labelFrame.pack(fill="both")
        mySuccess = ttk.Label(labelFrame, text="Imported " + export_name)
        mySuccess.pack(pady=20)
    else:
        unpack_all_frames()
        clear_frame(labelUnselectFrame)
        labelUnselectFrame.pack(fill="both")
        myErr = ttk.Label(labelUnselectFrame, text="Please select file and continue again")
        myErr.pack(pady=20)

   

def process_obj():
    make_directory()
    unpack_all_frames()
    openfile()

def convert_to_ply():
    all_name = ''
    # put the location to the folder where the plys are located here in this fashion
    path_to_obj_dir = os.path.join('3D Models')

    # get list of all files in directory
    file_list = sorted(os.listdir(path_to_obj_dir))

    # get a list of files ending in 'obj'
    ply_list = [item for item in file_list if item[-3:] == 'obj']
#     print(ply_list)
    unpack_all_frames()
    clear_frame(labelFrame)
    progressFrame.pack(fill="both")
    myprogress = ttk.Progressbar(progressFrame, orient= HORIZONTAL,length=300,mode='determinate')
    myprogress.pack(pady=20)
    myprogress['value'] += 100
    root.update_idletasks()
    bpy.ops.object.select_all(action='SELECT')
    bpy.ops.object.delete(use_global=False)
    
# loop through the strings in ply_list and add the files to the scene
    for item in ply_list:
        # full path and file name
        path_to_file = os.path.join(path_to_obj_dir, item)
        # import ply into the scene
        bpy.ops.import_scene.obj(filepath = path_to_file)
        bpy.context.selected_objects[0]
        # change suffix to ".obj"
        portion = os.path.splitext(item)
        newname = portion[0]+".ply"
        all_name += newname + " "
        # new full path and file name
        path_to_file2 = os.path.join(path_to_obj_dir, newname)
        # export current scene to an obj file
        bpy.ops.export_mesh.ply(filepath=path_to_file2, check_existing=True, filter_glob='*.ply', use_mesh_modifiers=False)
    #    bpy.ops.export_scene.obj(filepath = path_to_file2)
        # delete the object from the scene
        bpy.ops.object.delete()
    unpack_all_frames()
    labelFrame.pack(fill="both",expand=True)
    mySuccess = ttk.Label(labelFrame, text="Imported " + all_name)
    mySuccess.pack(pady=20)
    

def make_directory():
    path = "3D Models"
    # Check whether the specified path exists or not
    isExist = os.path.exists(path)
    if not isExist:
    # Create a new directory because it does not exist
       os.makedirs(path)
  
def clear_frame(frame):
   for widgets in frame.winfo_children():
      widgets.destroy()

def unpack_all_frames():
    frm.pack_forget()
    progressFrame.pack_forget()
    labelFrame.pack_forget()
    labelUnselectFrame.pack_forget()
    
menubar = Menu(root)
filemenu = Menu(menubar, tearoff=0)
filemenu.add_command(label="Decimate Images", command=process_obj)
filemenu.add_command(label="Convert Images", command=convert_to_ply)
filemenu.add_command(label="Exit", command=root.destroy)
menubar.add_cascade(label="3D images", menu=filemenu)

root.geometry("500x200")

root.config(menu=menubar)
frm = ttk.Frame(root, padding=10)
progressFrame = ttk.Frame(root,padding=10)
labelFrame = ttk.Frame(root, padding= 10)
labelUnselectFrame = ttk.Frame(root, padding= 10)
frm.pack(fill="both")
ttk.Label(frm, text="Decimate the 3D object file and import").pack()

root.mainloop()